<a href="https://colab.research.google.com/github/KauaHenSilva/python_nn_breast_cancer/blob/main/breast_cancer_registro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recuperar arquivos

In [ ]:
import os
tipo_de_recuperacao = "Ambos" #@param ["Download", "Drive", "Ambos"]
def repostirio_para_download():
  if os.path.exists('python_download_file_drive'):
    !cd python_download_file_drive && git pull
  else:
    !git clone https://github.com/KauaHenSilva/python_download_file_drive
  %pip install --upgrade --quiet -r python_download_file_drive/requirements.txt

## Recuperação por download

In [ ]:
if tipo_de_recuperacao in ["Download", "Ambos"]:
  from google.colab import auth
  from googleapiclient.discovery import build
  auth.authenticate_user()
  repostirio_para_download()
  !python "python_download_file_drive/main.py" "https://drive.google.com/drive/folders/183Mj7Uk4jLSbP46-UISygf3gmdp8Y-Pt?usp=drive_link" arquivos

Cloning into 'python_download_file_drive'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 57 (delta 29), reused 29 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 13.51 KiB | 4.50 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Baixando arquivos: 100% 2/2 [00:09<00:00,  4.76s/arquivo]


## Recuperação por drive

In [ ]:
# @markdown #### Caso escolheu o tipo de recuperação drive/ambos e não tenha os arquivos, faça um atalho desse drive:
# @markdown > https://drive.google.com/drive/folders/183Mj7Uk4jLSbP46-UISygf3gmdp8Y-Pt?usp=drive_link
path_atalho_drive = "/content/drive/MyDrive/Projetos Do GitHub/python_nn_breast_cancer" #@param {type:"string"}

In [ ]:
if tipo_de_recuperacao in ["Drive", "Ambos"]:
  from google.colab import drive
  drive.mount('/content/drive')
  if not os.path.exists(path_atalho_drive):
    raise Exception("O caminho especificado não existe no drive.")

Mounted at /content/drive


## Referenciar arquivo no drive ou download

In [ ]:
def get_referencia_valida(path):
    path_download = os.path.join("/content/arquivos", path)
    path_drive = os.path.join(path_atalho_drive, path)

    bool_download = os.path.exists(path_download)
    bool_drive = os.path.exists(path_drive)

    if not bool_download and not bool_drive:
        raise Exception(f"Arquivo não encontrado em nenhum dos caminhos (Download ou Drive). Caminhos: {path_download}, {path_drive}")

    if not bool_drive and bool_download:
        print("[INFO] Arquivo encontrado no download, mas não no drive.")
        return path_download

    if bool_drive and not bool_download:
        print("[INFO] Arquivo encontrado no drive, mas não no download.")
        return path_drive

    return path_download


# breast cancer tuning

In [ ]:
!pip install tensorflow==2.16.1 scikit-learn==1.5.0  scikeras==0.13.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 114.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.16.1 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.16.1 which is incompatible.


In [ ]:
import scikeras
import pandas as pd
import tensorflow as tf
import sklearn
import numpy as np

In [ ]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.2.2', '2.16.1', '1.5.0', '0.13.0')

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k
from sklearn.model_selection import GridSearchCV

In [ ]:
path_entrada = get_referencia_valida("entradas_breast.csv")
path_saida = get_referencia_valida("saidas_breast.csv")

In [ ]:
x = pd.read_csv(path_entrada)
y = pd.read_csv(path_saida)
x.shape, y.shape

((569, 30), (569, 1))

In [ ]:
rede_neural = Sequential()
rede_neural.add(tf.keras.layers.InputLayer(shape=(30,)))
rede_neural.add(tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
rede_neural.add(tf.keras.layers.Dropout(rate = 0.2))
rede_neural.add(tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
rede_neural.add(tf.keras.layers.Dropout(rate = 0.2))
rede_neural.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
rede_neural.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

In [ ]:
rede_neural.fit(x, y, 10, 100)

Epoch 1/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.5200 - loss: 2.0309
Epoch 2/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6954 - loss: 0.5609
Epoch 3/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7277 - loss: 0.4737
Epoch 4/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7302 - loss: 0.4654
Epoch 5/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7886 - loss: 0.4494
Epoch 6/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8159 - loss: 0.4307
Epoch 7/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8237 - loss: 0.4176
Epoch 8/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8435 - loss: 0.3719
Epoch 9/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8366 - loss: 0.4121
Epoch 10/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8807 - loss: 0.3603
Epoch 11/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8309 -

In [ ]:
x

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.12380,0.18660,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.14440,0.42450,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.00000,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.13740,205.00000,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,111.00000,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.45,26.40,166.10,2027.0,141.00000,0.21130,0.4107,0.2216,206.0000,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,144.00000,0.09791,0.1752,0.05533,...,23.69,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,159.0000,0.05648,...,18.98,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,277.00000,0.35140,152.00000,0.2397,0.07016,...,25.74,39.42,184.60,1821.0,165.00000,0.86810,0.9387,265.0000,0.4087,124.00000


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1[' radius_mean'].plot(kind='hist', bins=20, title=' radius_mean')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2[' texture_mean'].plot(kind='hist', bins=20, title=' texture_mean')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3[' perimeter_mean'].plot(kind='hist', bins=20, title=' perimeter_mean')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series[' area_mean']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel(' area_mean')

from matplotlib import pyplot as plt
_df_12['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13[' radius_mean'].plot(kind='line', figsize=(8, 4), title=' radius_mean')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14[' texture_mean'].plot(kind='line', figsize=(8, 4), title=' texture_mean')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15[' perimeter_mean'].plot(kind='line', figsize=(8, 4), title=' perimeter_mean')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
novo = np.array([[ 15.80,  8.34,   118,   900,  0.10,  0.26,  0.08, 0.134, 0.178,  0.20,
                    0.05,  1098,  0.87,  4500, 145.2, 0.005,  0.04,  0.05, 0.015,  0.03,
                   0.007, 23.15, 16.64, 178.5,  2018,  0.14, 0.185,  0.84,   158, 0.363]])

In [ ]:
previsao = rede_neural.predict(novo)
previsao

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


array([[1.]], dtype=float32)

In [ ]:
if previsao > 0.5:
  print("Maligno")
else:
  print("Benigno")

Maligno


# Save and Load

In [ ]:
rede_neural.save('classificador_breast.keras')

In [ ]:
nova = tf.keras.models.load_model("/content/classificador_breast.keras")